In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Installing the diffusers library, which is a popular library for diffusion models (generative models).
# Using pip, the package installer for Python, to download and install the 'diffusers' package from PyPI (Python Package Index).
# The 'diffusers' library includes various tools and models related to generative art, particularly focusing on Stable Diffusion and other diffusion-based models.

pip install diffusers


In [ ]:
from diffusers import StableDiffusionPipeline

In [ ]:
# Importing the torch library, which provides functions for tensor computation and deep learning operations.
import torch

# Importing the os library for interacting with the operating system, especially for file and directory operations.
import os

# Importing DataLoader from torch.utils.data, which is used to load datasets in batches for efficient training.
from torch.utils.data import DataLoader

# Importing datasets and transforms from torchvision, which provides standard datasets and image transformation utilities.
from torchvision import datasets, transforms

# Importing the StableDiffusionPipeline class from diffusers, which allows to load and work with a pre-trained Stable Diffusion model.
from diffusers import StableDiffusionPipeline  # Correct import

# Importing tqdm, a progress bar utility for Python loops to visualize the processing status.
from tqdm import tqdm

# Importing AdamW, an optimizer class from torch.optim, which implements the Adam optimizer with weight decay.
from torch.optim import AdamW

# Importing nn from torch, which provides a set of neural network layers and utilities for model building.
import torch.nn as nn

# Importing the Image class from the Python Imaging Library (PIL), used for opening, manipulating, and saving image files.
from PIL import Image


In [ ]:
class StableDiffusionFinetuner:
    def __init__(self, 
                 model_name="stabilityai/stable-diffusion-2-1-base", 
                 epochs=10,  # Number of training epochs
                 batch_size=4,  # Batch size for training
                 output_dir="/kaggle/working/fine_tuned_images",  # Output directory to save images
                 lr=1e-5):  # Learning rate for fine-tuning
        """
        Initializes fine-tuning with specified parameters.
        """
        # Checking if a GPU is available. If not, it defaults to CPU.
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Setting the fine-tuning parameters like model name, epochs, batch size, output directory, and learning rate.
        self.model_name = model_name
        self.epochs = epochs
        self.batch_size = batch_size
        self.output_dir = output_dir
        self.lr = lr

        # Ensuring the output directory exists or creating it if it does not.
        os.makedirs(self.output_dir, exist_ok=True)

        # Loading the pre-trained model using the load_model method.
        self.pipe = self.load_model()

        # Preparing the data loader with the prepare_data method to handle training data.
        self.dataloader = self.prepare_data()

        # Creating the optimizer using AdamW, which is a commonly used optimizer for fine-tuning.
        self.optimizer = AdamW(self.pipe.unet.parameters(), lr=self.lr)
        
        # Setting the loss function (MSE Loss) which is commonly used in generative models for fine-tuning.
        self.criterion = nn.MSELoss()

    def load_model(self):
        """Load the pre-trained Stable Diffusion model."""
        try:
            # Loading the pre-trained Stable Diffusion model using the `from_pretrained` method from the diffusers library.
            pipe = StableDiffusionPipeline.from_pretrained(self.model_name).to(self.device)
            return pipe
        except Exception as e:
            # Handling any errors that may occur during model loading and raising an exception if needed.
            print(f"Error loading model: {e}")
            raise

    def prepare_data(self):
        """Prepare the CIFAR-10 dataset for fine-tuning."""
        # Defining the data transformation pipeline to resize images, convert them to tensors, and normalize them.
        transform = transforms.Compose([
            transforms.Resize((512, 512)),  # Resizing images to 512x512 as required by the Stable Diffusion model
            transforms.ToTensor(),  # Converting images to tensors for model input
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizing based on ImageNet stats
        ])
        
        # Loading the CIFAR-10 dataset with the specified transformations.
        cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        
        # Creating a DataLoader for batching and shuffling the dataset.
        dataloader = DataLoader(cifar10, batch_size=self.batch_size, shuffle=True)
        
        return dataloader

    def generate_images(self, epoch, batch_idx, img, label):
        """
        Generate images using image-to-image transformation.
        """
        try:
            # Converting the image tensor into a PIL image format for use with the model.
            img_pil = transforms.ToPILImage()(img.cpu())
    
            # Generating the prompt based on the class label for image-to-image transformation.
            class_name = self.dataloader.dataset.classes[label.item()]
            img2img_prompt = f"Enhanced, photorealistic version of a {class_name}"
    
            # Generating an image by passing the prompt and input image to the Stable Diffusion model.
            output = self.pipe(
                prompt=img2img_prompt, 
                init_image=img_pil,  # Input image for transformation
                strength=0.75,  # The strength controls how much the original image is modified
                num_inference_steps=50,  # Number of steps to refine the image
                guidance_scale=7.5  # Controls how strongly the model follows the prompt
            )
            
            # Ensuring that the output contains at least one generated image.
            img2img_generated = output.images[0]  # Taking the first generated image
    
            # Saving the generated image to the specified output directory.
            img2img_save_path = os.path.join(self.output_dir, f"img2img_gen_epoch_{epoch+1}_batch_{batch_idx+1}.png")
            img2img_generated.save(img2img_save_path)
            print(f"Saved generated image: {img2img_save_path}")
    
            return img2img_generated
    
        except Exception as e:
            # Handling any errors that may occur during the image generation process.
            print(f"Error during image generation: {e}")
            return None

    def fine_tune(self):
        """Fine-tune the model over multiple epochs."""
        print("Starting fine-tuning...")

        # Looping over the number of epochs to fine-tune the model.
        for epoch in range(self.epochs):
            print(f"Epoch {epoch + 1}/{self.epochs}")

            # Using a progress bar (from tqdm) to track the progress of each epoch.
            progress_bar = tqdm(self.dataloader, desc=f"Epoch {epoch + 1}", unit="batch")

            # Looping through the data in batches for each epoch.
            for batch_idx, (img, label) in enumerate(progress_bar):
                try:
                    # Moving the data to the GPU (if available).
                    img = img.to(self.device)
                    label = label.to(self.device)

                    # Generating images for the current batch using the image-to-image method.
                    generated_img = self.generate_images(epoch, batch_idx, img[0], label[0])

                    if generated_img is not None:
                        # Calculating the loss between the generated image and the original image.
                        loss = self.criterion(generated_img, img[0])

                        # Performing backpropagation by zeroing gradients, computing gradients, and updating the weights.
                        self.optimizer.zero_grad()  # Clearing the previous gradients
                        loss.backward()  # Computing gradients for the loss
                        self.optimizer.step()  # Updating the model weights

                        # Updating the progress bar with the current batch loss.
                        progress_bar.set_postfix({"Batch": batch_idx + 1, "Loss": loss.item()})

                except Exception as e:
                    # Handling errors that may occur during the processing of each batch.
                    print(f"Error in batch {batch_idx}: {e}")

            # Printing the loss after each epoch to track fine-tuning progress.
            print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

        # Notifying the user that fine-tuning is completed.
        print("Fine-tuning completed.")


